In [30]:
import agentpy as ap
import random
import IPython
import numpy as np

# For using ggplot from r for better figures
from pyper import *
import os
r = R(RCMD="/Library/Frameworks/R.framework/Resources/bin/R", use_pandas=True)

Implements a basic agent-based version of the Deffuant opinion dynamics model using agentspy. Fully mixed population (non-spatial and non-topological).

In [31]:
class Person(ap.Agent):

    def setup(self):
        """ Initialise new agent variables at agent creation """

        self.opinion = random.uniform(-1,1)
        self.uncertainty = np.random.beta(self.p.alpha, self.p.beta)
        self.new_opinion = self.opinion # Used for synchronous updating

    def opinion_influence(self):
        """ This agent influences other agents in the population """

        # Randomly select a fixed number of agents as partners to influence
        partners = self.model.pop.random(self.p.num_partners, replace=False)
        for partner in partners:
            opinion_diff = abs(self.opinion - partner.opinion)
            if opinion_diff <= self.uncertainty:
                opinion_change = self.p.mu * (self.opinion - partner.opinion)
                partner.new_opinion = partner.opinion + opinion_change

    def update_opinion(self):
        self.opinion = self.new_opinion

class OpinionModel(ap.Model):

    def setup(self):
        """ Initialise the model and setup the agent population """
        
        # Create agent population
        self.pop = ap.AgentList(self, self.p.pop_size, Person)

    def update(self):
        """ Called after setup and every step """

        # Synchronous updating of opinions
        self.pop.update_opinion()
        # Record stats
        self.pop.record("opinion")

    def step(self):
        """ Model events per timestep """

        self.pop.opinion_influence()


In [32]:
parameters = {
    'pop_size': 200,
    'steps': 500,
    'num_partners': 1,
    'mu': 0.1,
    'max_uncertainty': 0.25,
    'alpha': 3,
    'beta': 6
}

In [33]:
model = OpinionModel(parameters)
results = model.run()

Completed: 500 steps
Run time: 0:00:00.699722
Simulation finished


In [34]:
# Get all logged data for Person agents
results = results.variables.Person
# Convert the resulting Pandas series to a dataFrame
results = results.opinion.to_frame().reset_index()

In [36]:
# Use R for plotting because of ggplot and dplyr
r.assign("Rresults",results)
r("""
library(ggplot2)
ggplot(Rresults,aes(x=t,y=opinion,colour=factor(obj_id))) +
      geom_line(size=0.6,alpha=0.6) +
      theme_minimal() +
      theme(legend.position="none")
ggsave("test99.png")
""")
os.system("open test99.png")

0